In [1]:
# let's make a start from here
! python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 15 kB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 8.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b2783e12d0f44606b10a66243066d3d46f4b6920b19957e2d1a51df6f8b88142
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [12]:
import pandas as pd
import spacy

nlp =  spacy.load('ru_core_news_lg')
df = pd.read_csv('/content/test_data.csv')
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "HI", "pattern": "здравствуйте"}, 
            {"label": "HI", "pattern": "привет"},
            {"label": "HI", "pattern": "добрый день"},
            {"label": "BYE", "pattern": "до свидания"},
            {"label": "BYE", "pattern": "всего доброго"},
            {"label": "ORG", "pattern": "диджитал бизнес"},
            {"label": "ORG", "pattern": "китобизнес"},
            {"label": "PER", "pattern": "анастасия"}
            ]
ruler.add_patterns(patterns)

for item in df.dlg_id.unique():
  txt = '. '.join(df[(df['dlg_id']== item) & (df['role']=='manager')].text)
  doc = nlp(txt)
  print(f'{item}-й диалог:')
  flag_name = flag_org = flag_hi = flag_bye = 0
  for ent in doc.ents:

    if ent.label_ == 'HI':
      start = txt.rfind('.', 0, ent.start_char) 
      if start == -1: 
        start = 0
      finish = txt.find('.', ent.end_char)
      flag_hi = 1
      print(f'реплика с приветствием: {txt[start: finish]}')

    if ent.label_ == 'PER' and ent.start_char < 100 and not flag_name:
      start = txt.rfind('.', 0, ent.start_char) 
      finish = txt.find('.', ent.end_char)
      print(f'реплика, где менеджер представил себя: {txt[start+1: finish]}')
      print(f'имя менеджера: {txt[ent.start_char: ent.end_char]}')
      flag_name = 1

    if ent.label_ == 'ORG' and ent.start_char < 100 and not flag_org:
      start = txt.rfind('.', 0, ent.start_char) 
      finish = txt.find('.', ent.end_char)
      print(f'название компании: {txt[ent.start_char: ent.end_char]}')
      flag_org = 1

    if ent.label_ == 'BYE':
      start = txt.rfind('.', 0, ent.start_char)
      start = 0 if start == -1 else start + 1
      finish = txt.find('.', ent.end_char)
      txt_out = txt[start:] if finish == -1 else txt[start:finish] 
      flag_bye = 1
      print(f'реплики, где менеджер попрощался: {txt_out}')

  print(f'требование к менеджеру: {"соблюдено" if flag_hi and flag_bye else "не соблюдено"}\n')

0-й диалог:
реплика с приветствием: Алло здравствуйте
реплика, где менеджер представил себя:  Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
имя менеджера: ангелина
название компании: диджитал бизнес
реплики, где менеджер попрощался:  Всего хорошего до свидания
требование к менеджеру: соблюдено

1-й диалог:
реплика с приветствием: Алло здравствуйте
реплика, где менеджер представил себя:  Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
имя менеджера: ангелина
название компании: диджитал бизнес
требование к менеджеру: не соблюдено

2-й диалог:
реплика с приветствием: Алло здравствуйте
реплика, где менеджер представил себя:  Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
имя менеджера: ангелина
название компании: диджитал бизнес
требование к ме